In [2]:
from xml.etree import ElementTree
import pandas as pd 
#https://www.weather.go.kr/weather/lifenindustry/sevice_rss.jsp
#http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=109

In [4]:
tree = ElementTree.parse('rss.xml')

In [5]:
root = tree.getroot()

In [7]:
root

<Element 'rss' at 0x0000020EA9A85630>

In [22]:
data_list = []
for item in root.findall('channel/item/description/body/location/data'):
    tm_ef = item.find('tmEf').text #find 메서드로 element 탐색, text 속성으로 값 추출 
    tmn = item.find('tmn').text
    tmx = item.find('tmx').text
    wf = item.find('wf').text
    data = pd.DataFrame({
        '일시':[tm_ef],
        '최저기온' : [tmn],
        '최고기온' : [tmx],
        '날씨' : [wf],
    })
    data_list.append(data)
weather = pd.concat(data_list)
weather.head()

,일시,최저기온,최고기온,날씨
0,2021-07-15 00:00,25,33,구름많음
0,2021-07-15 12:00,25,33,구름많음
0,2021-07-16 00:00,24,33,구름많음
0,2021-07-16 12:00,24,33,구름많고 소나기
0,2021-07-17 00:00,24,32,구름많음


In [10]:
type(weather)

pandas.core.frame.DataFrame

In [11]:
weather.to_csv('weather.csv') #csv로 저장

In [12]:
excel = pd.ExcelWriter('weather.xlsx')
weather.to_excel(excel, '.', index=False )
excel.save() #xlsx로 저장 

In [13]:
weather.reset_index(drop=True, inplace=True)

In [21]:
weather.head()

,일시,최저기온,최고기온,날씨
0,2021-07-15 00:00,25,33,구름많음
1,2021-07-15 12:00,25,33,구름많음
2,2021-07-16 00:00,24,33,구름많음
3,2021-07-16 12:00,24,33,구름많고 소나기
4,2021-07-17 00:00,24,32,구름많음


In [15]:
weather.to_json('weather.json') #json으로 저장 

In [16]:
import sqlite3
from pandas.io import sql
import os

In [17]:
with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
    try:
        weather.to_sql(name = 'WEATHER_INFO', con = con, index = False, if_exists='append') 
        #if_exists : {'fail', 'replace', 'append'} default : fail
    except Exception as e:
        print(str(e))
    
    query = 'SELECT * FROM WEATHER_INFO'
    data1 = pd.read_sql(query, con = con)

In [20]:
data1.head()

,일시,최저기온,최고기온,날씨
0,2021-07-15 00:00,25,33,구름많음
1,2021-07-15 12:00,25,33,구름많음
2,2021-07-16 00:00,24,33,구름많음
3,2021-07-16 12:00,24,33,구름많고 소나기
4,2021-07-17 00:00,24,32,구름많음


In [19]:
excel = pd.ExcelWriter('weather2.xlsx')
data1.to_excel(excel, '.', index=False )
excel.save()

In [23]:
df = pd.read_excel('weather2.xlsx')
df.head()

,일시,최저기온,최고기온,날씨
0,2021-07-15 00:00,25,33,구름많음
1,2021-07-15 12:00,25,33,구름많음
2,2021-07-16 00:00,24,33,구름많음
3,2021-07-16 12:00,24,33,구름많고 소나기
4,2021-07-17 00:00,24,32,구름많음
